In [1]:
# TODO DELETE ME
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# TODO

# find accuracy of Base model (using Spacy Root Verb Dependency)
# find accuracy using vanilla CRF Suite
# explore features and hyperparameters of CRF Suite Algo
# find accuracy using tweaked CRF Suite
# display results using Displacy
# interpret results using eli5

<IPython.core.display.Javascript object>

In [3]:
import warnings

warnings.filterwarnings("ignore")

<IPython.core.display.Javascript object>

In [4]:
import pandas as pd
import math

# to set random seed
import numpy as np

# used to create CRF model
from sklearn_crfsuite import CRF

# used to evaluate model
from sklearn_crfsuite import metrics

# for hyperparameter tuning parameter range
import scipy

# for specifying f1 metrics
from sklearn.metrics import make_scorer

# for cross validation of hyperparameters
from sklearn.model_selection import RandomizedSearchCV

# to visualize the weight of parameters of the fitted model
import eli5

Using TensorFlow backend.


<IPython.core.display.Javascript object>

In [10]:
pd.set_option("max_row", 600)

<IPython.core.display.Javascript object>

In [11]:
np.random.seed(42)

<IPython.core.display.Javascript object>

In [7]:
data = pd.read_csv("tagged_data.csv", index_col=0)

<IPython.core.display.Javascript object>

In [8]:
data.head(10)

,sentence#,word,pos,tag
0,0.0,Preheat,VB,U-Action
1,0.0,oven,NN,U-Utensil
2,0.0,to,IN,O
3,0.0,425,CD,O
4,0.0,degrees,NNS,O
5,0.0,F.,NN,O
6,1.0,Press,NN,U-Action
7,1.0,dough,NN,O
8,1.0,into,IN,O
9,1.0,the,DT,O


<IPython.core.display.Javascript object>

In [27]:
#################################################
# Dependency parser (skip)
###################################################

import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_sm")

<IPython.core.display.Javascript object>

In [28]:
# Locate validation set from sentence#212 onward (ID:212 to ID:263)
parse_data = data.iloc[2770:3445]

# Inspect the dataset
parse_data.tail()

,sentence#,word,pos,tag
3440,262.0,.,.,O
3441,263.0,Freeze,VB,U-Action
3442,263.0,and,CC,O
3443,263.0,enjoy,VB,O
3444,263.0,!,.,O


<IPython.core.display.Javascript object>

In [29]:
# Set the sentence# as index for merging ID:212 to ID:263
parse_data.set_index("sentence#", inplace=True)
parse_data.tail()

,word,pos,tag
sentence#,,,
262.0,.,.,O
263.0,Freeze,VB,U-Action
263.0,and,CC,O
263.0,enjoy,VB,O
263.0,!,.,O


<IPython.core.display.Javascript object>

In [30]:
# Merge the words with same sentence#
combined_sentence = (
    parse_data.groupby(["sentence#"])["word"]
    .apply(lambda x: " ".join(x.astype(str)))
    .reset_index()
)
combined_sentence.head()

,sentence#,word
0,212.0,Spray baking sheet with non - stick spray .
1,213.0,Spread potatoes onto baking sheet .
2,214.0,Bake for 15 minutes ; toss and turn ; bake for...
3,215.0,Serve immediately .
4,216.0,"In 1/4 cup butter , saute carrots , onion , ce..."


<IPython.core.display.Javascript object>

In [40]:
# Convert the dataframe to the nested list
combined_sentences_list = combined_sentence.reset_index()[
    ["sentence#", "word"]
].values.tolist()
for item in combined_sentences_list:
    print(item[1])

Spray baking sheet with non - stick spray .
Spread potatoes onto baking sheet .
Bake for 15 minutes ; toss and turn ; bake for five minutes ; add herbs and toss ; bake a final five minutes until the potatoes are brown and crisp .
Serve immediately .
In 1/4 cup butter , saute carrots , onion , celery and broccoli stems for 5 minutes .
Add thyme , oregano and basil ; saute 5 minutes more .
Add wine and deglaze pan .
Add hot chicken stock and reduce by one - third .
Add Worcestershire sauce , Tabasco , smoked chicken , beans and broccoli florets ; simmer 5 minutes .
Add cream , simmer 5 minutes more and season to taste ( thicken with cornstarch if desired ) .
Drop in remaining butter , piece by piece , stirring until melted and serve immediately .
Smoked Chicken : On a covered grill , slightly smoke boneless chicken , cooking to medium rare ( about 30 minutes ) .
Chef Meskan uses applewood chips and does not allow the grill to become too hot .
Preheat oven to 350 degrees .
Grease 13 x 9 b

<IPython.core.display.Javascript object>

In [44]:
# print(combined_sentence_list)
dp_picked_sent = []
not_dp_picked_sent  = []

# Get the sentence for SpaCy
for sentence in combined_sentences_list:
    # Locate the sentence element
    ind_sentences = sentence[1]
    # Create SpaCy doc
    doc = nlp(ind_sentences)

    # Get the action words
    for token in doc:
        # print(token)

        # Pick all the Verb in pos_, and anything but advcl in dep_
        if token.pos_ == "VERB" and token.dep_ != "advcl":
            # print(int(sentence[0]), " ".join([token.dep_ for token in doc]))
            dp_parser = " ".join([token.dep_ for token in doc])
            dp_picked_sent.append((int(sentence[0]), dep_parser))
        ###################################################
        # HELP: How to pick up the rest of missed setence ID and setnece itself
        # else:
        #      not_dp_picked_sent.append(sentence[0])
        #####################################################
        # if token.dep_ == "ROOT" or token.dep_ == "conj" or token.dep_ == "nmod" or token.dep_ == "advcl":
        # print(doc, "---->>>")

<IPython.core.display.Javascript object>

In [48]:
# Summary
print(
    "Total # of sentences:",
    len(combined_sentences_list),
    "\n# of Dependency Parser PICKED sentences:",
    len(set(dp_picked_sent)),
    "\nMISSED sentences ID:", 212, 213, 217, 226, 233, 259
)

Total # of sentences: 52 
# of Dependency Parser PICKED sentences: 46 
MISSED sentences ID: 212 213 217 226 233 259


<IPython.core.display.Javascript object>

In [21]:
doc1 = nlp(
    "Remove from freezer and place waxed paper between each burger as you stack them ."
)
displacy.serve(doc1, style="dep")


Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


<IPython.core.display.Javascript object>

In [17]:
##########################
# The end
##########################

<IPython.core.display.Javascript object>

In [9]:
data["tag"].value_counts()

O               2498
U-Action         366
U-Ingredient     241
B-Ingredient      70
L-Ingredient      70
B-Utensil         60
U-Utensil         59
L-Utensil         59
I-Utensil         15
I-Ingredient       3
L-Action           2
B-Action           2
Name: tag, dtype: int64

<IPython.core.display.Javascript object>

In [10]:
print(len(data["word"].values))

3445


<IPython.core.display.Javascript object>

In [11]:
words = list(set(data["word"].values))
len(words)

869

<IPython.core.display.Javascript object>

In [12]:
agg_func = lambda s: [
    (w, p, t)
    for w, p, t in zip(
        s["word"].values.tolist(), s["pos"].values.tolist(), s["tag"].values.tolist()
    )
]

<IPython.core.display.Javascript object>

In [13]:
grouped = data.groupby("sentence#").apply(agg_func)

<IPython.core.display.Javascript object>

In [14]:
sentences = [s for s in grouped]

<IPython.core.display.Javascript object>

In [15]:
len(sentences)

264

<IPython.core.display.Javascript object>

In [16]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        # set the bias for each word
        "bias": 1.0,
        # get the lower case form of the word
        "word.lower()": word.lower(),
        # get last 3 letters for the word
        "word[-3:]": word[-3:],
        # get last 2 letters for the word
        "word[-2:]": word[-2:],
        # check whether the word is uppercase or not
        "word.isupper()": word.isupper(),
        # check whether the word is title case or not
        "word.istitle()": word.istitle(),
        # check whether the word is digit or not, useful to identifying quantities which will be tagged as 'O'
        "word.isdigit()": word.isdigit(),
        # specifying the pos for word
        "postag": postag,
        # get first 2 letters for the POS tag
        "postag[:2]": postag[:2],
    }

    # if word is starting of sentence
    if i > 0:

        # if word is not the beginning of sentence
        # then get the word before it i.e. i-1 index
        word1 = sent[i - 1][0]

        # then get the pos before it i.e. i-1 index
        postag1 = sent[i - 1][1]

        features.update(
            {
                # setting the lower form of word at index i-1
                "-1:word.lower()": word1.lower(),
                # checking if the word at index i-1 is titlecase
                "-1:word.istitle()": word1.istitle(),
                # checking if the word at index i-1 is uppercase
                "-1:word.isupper()": word1.isupper(),
                # setting the pos of word at index i-1
                "-1:postag": postag1,
                # get first 2 letters for the POS tag for i-1 indexed word
                "-1:postag[:2]": postag1[:2],
            }
        )
    else:
        # setting the BOS or Begining of sentence to True
        features["BOS"] = True

    # if word is at the end of sentence
    if i < len(sent) - 1:

        # if word is not the end of sentence
        # then get the word after it i.e. i+1 index
        word1 = sent[i + 1][0]

        # then get the pos after it i.e. i+1 index
        postag1 = sent[i + 1][1]

        features.update(
            {
                # setting the lower form of word at index i+1
                "+1:word.lower()": word1.lower(),
                # checking if the word at index i+1 is titlecase
                "+1:word.istitle()": word1.istitle(),
                # checking if the word at index i+1 is titlecase
                "+1:word.isupper()": word1.isupper(),
                # setting the pos of word at index i+1
                "+1:postag": postag1,
                # get first 2 letters for the POS tag for i+1 indexed word
                "+1:postag[:2]": postag1[:2],
            }
        )
    else:
        # setting the EOS or End of sentence to True
        features["EOS"] = True

    return features

<IPython.core.display.Javascript object>

In [17]:
def sent2features(sent):
    """Convert sentences which are lists containing (w, p, t) into features"""
    return [word2features(sent, i) for i in range(len(sent))]

<IPython.core.display.Javascript object>

In [18]:
def sent2labels(sent):
    """Retrieve all the labels from sentences which are lists containing (w, p, t)"""
    return [label for token, postag, label in sent]

<IPython.core.display.Javascript object>

In [19]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

<IPython.core.display.Javascript object>

In [20]:
# split into train and test
boundary = math.ceil(len(X) * 0.8)

<IPython.core.display.Javascript object>

In [21]:
boundary

212

<IPython.core.display.Javascript object>

In [22]:
# train data
x_train = X[:boundary]
y_train = y[:boundary]

# test data
x_test = X[boundary:]
y_test = y[boundary:]

<IPython.core.display.Javascript object>

In [23]:
print(len(x_train))
print(len(x_test))

212
52


<IPython.core.display.Javascript object>

In [24]:
# creating CRF model with Gradient Descent
crf = CRF(
    algorithm="lbfgs",
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=False,
)

<IPython.core.display.Javascript object>

In [25]:
# fitting the model using train data
crf.fit(x_train, y_train)

CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=False,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

<IPython.core.display.Javascript object>

In [26]:
# to get all the labels/tags of data
labels = list(crf.classes_)

<IPython.core.display.Javascript object>

In [27]:
labels

['U-Action',
 'U-Utensil',
 'O',
 'B-Utensil',
 'L-Utensil',
 'U-Ingredient',
 'B-Ingredient',
 'L-Ingredient',
 'I-Utensil',
 'I-Ingredient',
 'B-Action',
 'L-Action']

<IPython.core.display.Javascript object>

Since we are not interested in 'O' tags we will check the performance of the CRF model using f1 scores for every tag except O tags.

In [28]:
labels.remove("O")
labels

['U-Action',
 'U-Utensil',
 'B-Utensil',
 'L-Utensil',
 'U-Ingredient',
 'B-Ingredient',
 'L-Ingredient',
 'I-Utensil',
 'I-Ingredient',
 'B-Action',
 'L-Action']

<IPython.core.display.Javascript object>

In [29]:
# performing predictions based on the fitted model
y_pred = crf.predict(x_test)

<IPython.core.display.Javascript object>

In [30]:
# finding the f1 score
metrics.flat_f1_score(y_test, y_pred, average="weighted", labels=labels)

0.719570446804729

<IPython.core.display.Javascript object>

In [31]:
# finding the f1 score
print(metrics.flat_classification_report(y_test, y_pred, labels=labels, digits=3))

              precision    recall  f1-score   support

    U-Action      0.911     0.878     0.894        82
   U-Utensil      1.000     0.412     0.583        17
   B-Utensil      1.000     0.231     0.375        13
   L-Utensil      1.000     0.308     0.471        13
U-Ingredient      0.738     0.689     0.713        45
B-Ingredient      0.474     0.750     0.581        12
L-Ingredient      0.474     0.750     0.581        12
   I-Utensil      1.000     0.333     0.500         3
I-Ingredient      0.000     0.000     0.000         0
    B-Action      0.000     0.000     0.000         0
    L-Action      0.000     0.000     0.000         0

   micro avg      0.782     0.690     0.733       197
   macro avg      0.600     0.395     0.427       197
weighted avg      0.839     0.690     0.720       197



<IPython.core.display.Javascript object>

The model overfits!!

In [32]:
len(data[((data["word"] == "Boil") | (data["word"] == "boil"))])

13

<IPython.core.display.Javascript object>

In [33]:
len(data[((data["word"] == "Cover") | (data["word"] == "cover"))])

17

<IPython.core.display.Javascript object>

In [34]:
len(data[((data["word"] == "Preheat") | (data["word"] == "preheat"))])

9

<IPython.core.display.Javascript object>

In [35]:
eli5.show_weights(crf, top=30)

From \ To,O,B-Action,L-Action,U-Action,B-Ingredient,I-Ingredient,L-Ingredient,U-Ingredient,B-Utensil,I-Utensil,L-Utensil,U-Utensil
O,3.17,0.363,0.0,1.63,1.197,0.0,0.0,2.178,2.136,0.0,0.0,1.323
B-Action,0.0,0.0,2.47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
L-Action,0.368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U-Action,1.472,0.0,0.0,0.0,0.738,0.0,0.0,1.544,0.0,0.0,0.0,1.47
B-Ingredient,0.0,0.0,0.0,0.0,0.0,1.447,5.673,0.0,0.0,0.0,0.0,0.0
I-Ingredient,-0.108,0.0,0.0,0.0,0.0,0.0,1.425,0.0,0.0,0.0,0.0,0.0
L-Ingredient,1.276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U-Ingredient,1.786,0.0,0.0,0.313,0.615,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B-Utensil,-1.275,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.279,4.216,0.0
I-Utensil,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.542,3.67,0.0


<IPython.core.display.Javascript object>

**Hyperparameter Optimization**

In [36]:
# creating a CRF Hyperparameter tuned model
crf_hp = CRF(
    algorithm="lbfgs", c1=10, c2=0.1, max_iterations=100, all_possible_transitions=False
)

# parameters to hypertune
params_space = {"c1": scipy.stats.expon(scale=0.5), "c2": scipy.stats.expon(scale=0.05)}

# metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score, average="weighted", labels=labels)

# search using 3 fold cross validation
rs = RandomizedSearchCV(
    crf_hp, params_space, cv=3, verbose=1, n_jobs=-1, n_iter=50, scoring=f1_scorer
)

<IPython.core.display.Javascript object>

In [ ]:
# fitting the hyperparameters
rs.fit(x_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   14.9s


Best Parameters:

In [ ]:
print("best params:", rs.best_params_)
print("best CV score:", rs.best_score_)

In [ ]:
# set the best estimator
crf_hp_tuned = rs.best_estimator_

In [ ]:
# predict using the best CRF model
y_pred = crf_hp_tuned.predict(x_test)

In [ ]:
# print the f1 evaluation metric
print(metrics.flat_classification_report(y_test, y_pred, labels=labels, digits=3))

In [ ]:
# transition features: given B-Utensil then what is score for I-Utensil

In [ ]:
# state features: what property of feature describes the tag

In [ ]:
eli5.show_weights(crf_hp_tuned, top=30)

## Untagged Data Stuff

In [ ]:
untagged_test_data = pd.read_csv("./Untagged Test Data/untagged_test_data.csv")

In [ ]:
untagged_test_data.head()

In [ ]:
len(untagged_test_data)

In [ ]:
# 30:5:30

In [ ]:
agg_func_test = lambda s: [
    (w, p) for w, p in zip(s["word"].values.tolist(), s["pos"].values.tolist())
]

In [ ]:
grouped_test = untagged_test_data.groupby(["recipe_name", "Step#"]).apply(agg_func_test)

In [ ]:
test_sentences = [s for s in grouped_test]

In [ ]:
len(test_sentences)

In [ ]:
X_untagged_test = [sent2features(s) for s in test_sentences]

In [ ]:
len(X_untagged_test)

In [ ]:
pred_test = crf_hp_tuned.predict(X_untagged_test)

In [ ]:
len(pred_test)

In [ ]:
count = 0
for l in pred_test:
    for tag in l:
        if tag == "U-Action":
            count += 1

In [ ]:
count

In [ ]:
len(list(untagged_test_data["word"].values))

In [ ]:
len(list(set(untagged_test_data["word"].values)))

Boostrap output to the untagged data df

In [ ]:
flat_list = []
for sublist in pred_test:
    for item in sublist:
        flat_list.append(item)

In [ ]:
len(flat_list)

In [ ]:
untagged_test_data["Predicted Output"] = flat_list

In [ ]:
len(untagged_test_data[untagged_test_data["Predicted Output"] == "U-Action"])

In [ ]:
untagged_test_data[untagged_test_data["Predicted Output"] == "U-Action"]